In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd
import math
import socket
import time
import csv

# Saving pandas dataframe as CSV

In [9]:
numberOfPeople = 1
num_coords = 17
landmarks = ['t']
for person in range(0, numberOfPeople):
    for coords in range(0, num_coords):
        landmarks += ['{}_y{}'.format(person, coords), '{}_x{}'.format(person, coords), '{}_s{}'.format(person, coords)]

In [10]:
csv_path = '../../data/csv/'
csv_name = 'actionPracticeGrandTruth.csv'
with open(csv_path + csv_name, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
### Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

### Load Video
video_path = '../../data/videos/'
actionPractice = video_path + 'actionPracticeSmall.mp4'

### Set CSV
csv_path = '../../data/csv/'

### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
# Variables for drawing plot in real-time


if __name__ == "__main__":
    ### Variables
#     numberOfPeople = 1    ### Change the number of people here!!
    
    start = time.time()

    ### Loading Video File
    cap = cv2.VideoCapture(actionPractice)    ### Change the File Here!!
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)
        
#         print(keypoints_with_scores)    # This is coordinates data.
#         print(time.time() - start)    # This is program running time.
        
        # Make coordinates flatten
        row = []
        for person in range(0, numberOfPeople):
            tempRow = list(np.array([[keypoint[0], keypoint[1], keypoint[2]] for keypoint in keypoints_with_scores[person]]).flatten())
            row.append(tempRow)
        
        row.sort(key=lambda x: (x[1], x[0]))    # sorted with order of people (from left to right)
        row = sum(row, [])    # Convert 2-dim List to flat

        # Append time series
        row.insert(0, time.time()-start)
        
        # Export to CSV
        with open(csv_path + csv_name, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row)


        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....

        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'copy'

# Filtered Data with FPS 3

In [20]:
# Read the original CSV file and filter the rows
filtered_rows = []
with open(csv_path + 'actionPracticeGrandTruth.csv', 'r') as input_file:
    reader = csv.reader(input_file)
    header = next(reader)  # Read and skip the header row

    for index, row in enumerate(reader):
        if index % 10 == 0:
            filtered_rows.append(row)

# Write the filtered rows to a new CSV file
with open(csv_path + 'actionPracticeGrandTruth_filtered.csv', 'w', newline='') as output_file:
    writer = csv.writer(output_file)
    writer.writerow(header)
    writer.writerows(filtered_rows)

print("Filtered data saved to 'filtered_data.csv'")

Filtered data saved to 'filtered_data.csv'
